In [ ]:
# ====== 0) Setup ======
!pip -q install pandas numpy matplotlib openpyxl > /dev/null

import os, base64
from typing import List, Optional, Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 140

# ====== 1) Config — set these two for each dataset run ======
dataset_choice = 'uci_online_retail'     # 'kaggle_talib' | 'kaggle_prasad' | 'uci_online_retail'
csv_path = '/content/Online Retail.xlsx'  # <-- path to the UCI Excel file

# Agg frequency for plots (monthly)
agg_freq = 'MS'

# ====== 2) Helpers to read, infer columns, and canonicalize ======
def _read_any(path: str) -> pd.DataFrame:
    """Read CSV or Excel seamlessly."""
    ext = os.path.splitext(path)[1].lower()
    if ext in ['.xlsx', '.xls']:
        # UCI file is xlsx; first sheet is fine
        return pd.read_excel(path, sheet_name=0, engine='openpyxl')
    else:
        return pd.read_csv(path, low_memory=False, encoding='utf-8', on_bad_lines='skip')

def _find_col(cands: List[str], cols: List[str]) -> Optional[str]:
    low = [c.lower() for c in cols]
    for cand in cands:
        for i, c in enumerate(low):
            if cand in c:
                return cols[i]
    return None

def _ensure_dt(s: pd.Series) -> pd.Series:
    if pd.api.types.is_datetime64_any_dtype(s):
        return s
    return pd.to_datetime(s, errors='coerce')

def load_and_canonicalize(csv_path: str, dataset_choice: str) -> Tuple[pd.DataFrame, Dict[str,str]]:
    df = _read_any(csv_path)
    cols = list(df.columns)

    # generic guesses (order matters)
    date_col     = _find_col(['date','invoice date','order date','datetime','invoicedate'], cols)
    qty_col      = _find_col(['quantity','qty','units','total_items','items'], cols)
    price_col    = _find_col(['unitprice','unit_price','price','unit cost'], cols)
    revenue_col  = _find_col(['revenue','sales','amount','total_cost','total price','total amount','total'], cols)
    region_col   = _find_col(['region','country','state','city','market'], cols)
    category_col = _find_col(['category','department','segment','class','store_type','customer_category'], cols)
    product_col  = _find_col(['product','sku','item','stockcode','description','name'], cols)

    # ---- Dataset-specific nudges ----
    if dataset_choice == 'uci_online_retail':
        # Standard UCI columns
        date_col     = date_col     or 'InvoiceDate'
        qty_col      = qty_col      or 'Quantity'
        price_col    = price_col    or 'UnitPrice'
        region_col   = region_col   or 'Country'
        product_col  = product_col  or 'StockCode'

        # Drop cancellations/returns and negative quantities
        if 'InvoiceNo' in df.columns:
            df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
        if 'Quantity' in df.columns:
            df = df[df['Quantity'] > 0]

        # Derive category from first word of Description if no category
        if category_col is None:
            desc = 'Description' if 'Description' in df.columns else product_col
            df['_derived_category'] = df[desc].astype(str).str.split().str[0].str.upper()
            category_col = '_derived_category'

        # Compute revenue if not present: Quantity * UnitPrice
        if revenue_col is None and qty_col in df and price_col in df:
            df['_derived_revenue'] = pd.to_numeric(df[qty_col], errors='coerce') * pd.to_numeric(df[price_col], errors='coerce')
            revenue_col = '_derived_revenue'

    # Generic fallback: compute revenue if we have qty & price but no revenue
    if revenue_col is None and (qty_col in df and price_col in df):
        df['_derived_revenue'] = pd.to_numeric(df[qty_col], errors='coerce') * pd.to_numeric(df[price_col], errors='coerce')
        revenue_col = '_derived_revenue'

    # ---- Build canonical ----
    if date_col is None:
        raise ValueError("Couldn't find a date column; rename your date to include 'date' (e.g., InvoiceDate).")

    canon = pd.DataFrame()
    canon['date'] = _ensure_dt(df[date_col])
    canon['quantity'] = pd.to_numeric(df[qty_col], errors='coerce') if qty_col in df else np.nan
    canon['revenue']  = pd.to_numeric(df[revenue_col], errors='coerce') if revenue_col in df else np.nan
    canon['region']   = df[region_col].astype(str) if region_col in df else 'UNKNOWN'
    canon['category'] = df[category_col].astype(str) if category_col in df else 'UNKNOWN'
    canon['product']  = df[product_col].astype(str) if product_col in df else 'UNKNOWN'
    canon = canon.dropna(subset=['date']).sort_values('date').reset_index(drop=True)

    mapping = dict(date=date_col, qty=qty_col, price=price_col, revenue=revenue_col,
                   region=region_col, category=category_col, product=product_col)
    return canon, mapping

# ====== 3) Plot utils ======
def _ensure_dir(p): os.makedirs(p, exist_ok=True)

def _savefig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=140, bbox_inches='tight')
    plt.close()

def monthly_agg(df, col='revenue'):
    m = (df.assign(month=df['date'].dt.to_period('M').dt.to_timestamp())
           .groupby('month', as_index=False)[col].sum())
    return m.rename(columns={'month':'date', col:'value'})

def heat_image(pivot_df, title, path):
    data = pivot_df.values.astype(float)
    fig, ax = plt.subplots(figsize=(10,4))
    im = ax.imshow(data, aspect='auto')
    ax.set_xticks(range(pivot_df.shape[1])); ax.set_xticklabels(list(pivot_df.columns), rotation=45, ha='right')
    ax.set_yticks(range(pivot_df.shape[0])); ax.set_yticklabels(list(pivot_df.index))
    ax.set_title(title); fig.colorbar(im, ax=ax)
    _savefig(path)

def pareto_bar(series, title, path, top=15):
    s = series.sort_values(ascending=False).head(top)
    cum = s.cumsum()/s.sum()*100
    fig, ax1 = plt.subplots(figsize=(10,4))
    x = np.arange(len(s))
    ax1.bar(x, s.values)
    ax1.set_xticks(x)
    ax1.set_xticklabels(s.index.astype(str), rotation=45, ha='right')
    ax1.set_ylabel('Revenue')
    ax2 = ax1.twinx()
    ax2.plot(x, cum.values, marker='o')
    ax2.set_ylabel('Cumulative %'); ax2.set_ylim(0, 110)
    ax1.set_title(title)
    _savefig(path)

def zscore_anomalies(series, window=6, thresh=2.0):
    x = series.copy()
    mu = x.rolling(window, min_periods=3).mean()
    sd = x.rolling(window, min_periods=3).std()
    z = (x - mu) / sd
    flags = (z.abs() >= thresh)
    return z, flags

# ====== 4) EDA generator ======
def make_eda(canon: pd.DataFrame, ds_name: str):
    out_dir = f'/content/eda_reports/{ds_name}'
    _ensure_dir(out_dir)

    # KPIs
    kpis = {
        'rows': len(canon),
        'date_min': canon['date'].min(),
        'date_max': canon['date'].max(),
        'total_revenue': float(canon['revenue'].sum(skipna=True)),
        'total_quantity': float(canon['quantity'].sum(skipna=True)) if 'quantity' in canon else float('nan'),
        'n_regions': canon['region'].nunique(),
        'n_categories': canon['category'].nunique(),
        'n_products': canon['product'].nunique(),
    }
    pd.DataFrame([kpis]).to_csv(os.path.join(out_dir, 'Summary.csv'), index=False)

    # 1) Monthly revenue
    mrev = monthly_agg(canon, 'revenue')
    plt.figure(figsize=(10,4)); plt.plot(mrev['date'], mrev['value'], marker='o')
    plt.title('Monthly Revenue'); plt.xlabel('Date'); plt.ylabel('Revenue')
    _savefig(os.path.join(out_dir, '01_monthly_revenue.png'))

    # 2) Monthly quantity
    if canon['quantity'].notna().any():
        mqty = monthly_agg(canon, 'quantity')
        plt.figure(figsize=(10,4)); plt.plot(mqty['date'], mqty['value'], marker='o')
        plt.title('Monthly Quantity'); plt.xlabel('Date'); plt.ylabel('Units')
        _savefig(os.path.join(out_dir, '02_monthly_quantity.png'))

    # 3) Revenue by category
    cat = canon.groupby('category')['revenue'].sum().sort_values(ascending=False)
    plt.figure(figsize=(10,4)); plt.bar(cat.index.astype(str)[:15], cat.values[:15])
    plt.title('Top Categories by Revenue'); plt.xticks(rotation=45, ha='right'); plt.ylabel('Revenue')
    _savefig(os.path.join(out_dir, '03_category_revenue.png'))

    # 4) Revenue by region
    reg = canon.groupby('region')['revenue'].sum().sort_values(ascending=False)
    plt.figure(figsize=(10,4)); plt.bar(reg.index.astype(str)[:15], reg.values[:15])
    plt.title('Top Regions by Revenue'); plt.xticks(rotation=45, ha='right'); plt.ylabel('Revenue')
    _savefig(os.path.join(out_dir, '04_region_revenue.png'))

    # 5) Category x Month heatmap
    cm = (canon.assign(Month=canon['date'].dt.to_period('M').astype(str))
                .groupby(['category','Month'])['revenue'].sum().unstack(fill_value=0))
    cm = cm.loc[cm.sum(axis=1).sort_values(ascending=False).index[:12]]  # top 12 cats for readability
    heat_image(cm, 'Revenue Heatmap: Category x Month', os.path.join(out_dir, '05_cat_month_heat.png'))

    # 6) Top products by revenue
    prod = canon.groupby('product')['revenue'].sum().sort_values(ascending=False).head(15)
    plt.figure(figsize=(10,4)); plt.bar(prod.index.astype(str), prod.values)
    plt.title('Top 15 Products by Revenue'); plt.xticks(rotation=45, ha='right'); plt.ylabel('Revenue')
    _savefig(os.path.join(out_dir, '06_top_products.png'))

    # 7) Pareto of categories
    pareto_bar(cat, 'Pareto: Category Revenue Share', os.path.join(out_dir, '07_pareto_category.png'))

    # 8) MoM growth %
    mrev2 = mrev.copy()
    mrev2['mom_%'] = mrev2['value'].pct_change()*100
    plt.figure(figsize=(10,4)); plt.plot(mrev2['date'], mrev2['mom_%'], marker='o')
    plt.axhline(0, linestyle='--')
    plt.title('Month-over-Month Growth % (Revenue)'); plt.xlabel('Date'); plt.ylabel('%')
    _savefig(os.path.join(out_dir, '08_mom_growth.png'))

    # 9) Rolling mean vs actual
    mrev3 = mrev.copy(); mrev3['roll3'] = mrev3['value'].rolling(3, min_periods=1).mean()
    plt.figure(figsize=(10,4))
    plt.plot(mrev3['date'], mrev3['value'], label='Actual')
    plt.plot(mrev3['date'], mrev3['roll3'], label='3-mo Avg')
    plt.legend(); plt.title('Revenue: Actual vs 3-month Avg'); plt.xlabel('Date'); plt.ylabel('Revenue')
    _savefig(os.path.join(out_dir, '09_rolling_avg.png'))

    # 10) Simple anomaly flags (z-score on monthly revenue)
    z, flags = zscore_anomalies(mrev['value'])
    plt.figure(figsize=(10,4))
    plt.plot(mrev['date'], mrev['value'], label='Revenue')
    plt.scatter(mrev['date'][flags], mrev['value'][flags], c='red', label='Anomaly')
    plt.legend(); plt.title('Revenue with Simple Z-score Anomaly Flags'); plt.xlabel('Date'); plt.ylabel('Revenue')
    _savefig(os.path.join(out_dir, '10_anomalies.png'))

    # 11) Missing values per column
    miss = canon.isna().sum()
    plt.figure(figsize=(8,4)); plt.bar(miss.index.astype(str), miss.values)
    plt.title('Missing Values per Column'); plt.xticks(rotation=45, ha='right'); plt.ylabel('Count')
    _savefig(os.path.join(out_dir, '11_missing_values.png'))

    # 12) Revenue distribution
    plt.figure(figsize=(8,4)); plt.hist(canon['revenue'].dropna(), bins=30)
    plt.title('Revenue Distribution'); plt.xlabel('Revenue (row-level)'); plt.ylabel('Frequency')
    _savefig(os.path.join(out_dir, '12_revenue_histogram.png'))

    # ---- Build a tiny HTML report ----
    def img_tag(path):
        with open(path, 'rb') as f:
            b64 = base64.b64encode(f.read()).decode('utf-8')
        return f'<img src="data:image/png;base64,{b64}" style="max-width:100%;height:auto;"/><hr/>'

    html = f"""
    <html><head><meta charset="utf-8"><title>EDA Report - {ds_name}</title></head>
    <body>
    <h1>EDA Report - {ds_name}</h1>
    <p><b>Rows:</b> {kpis['rows']} | <b>Date Range:</b> {kpis['date_min']} → {kpis['date_max']} |
       <b>Total Revenue:</b> {kpis['total_revenue']:,.2f} |
       <b>Regions:</b> {kpis['n_regions']} | <b>Categories:</b> {kpis['n_categories']} | <b>Products:</b> {kpis['n_products']}</p>
    """
    for i in range(1,13):
        name = f"{i:02d}_"
        pics = sorted([p for p in os.listdir(out_dir) if p.startswith(name)])
        if not pics:  # skip if this plot wasn't generated (e.g., no quantity column)
            continue
        fpath = pics[0]
        html += f"<h3>{fpath.split('_',1)[1].replace('.png','').replace('_',' ').title()}</h3>"
        html += img_tag(os.path.join(out_dir, fpath))
    html += "</body></html>"
    with open(os.path.join(out_dir, 'EDA_Report.html'), 'w') as f:
        f.write(html)

    print(f"Done. See: {out_dir}")

# ====== 5) Run for the current dataset ======
canon, mapping = load_and_canonicalize(csv_path, dataset_choice)
print("Detected columns:", mapping)
make_eda(canon, ds_name=dataset_choice)


Detected columns: {'date': 'InvoiceDate', 'qty': 'Quantity', 'price': 'UnitPrice', 'revenue': '_derived_revenue', 'region': 'Country', 'category': '_derived_category', 'product': 'StockCode'}
Done. See: /content/eda_reports/uci_online_retail
